In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !ls /content/drive/My\ Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/inputs/ppe_input_images/ | wc -l

In [ ]:
# !ls /content/drive/My\ Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/ground_truths/ppe_gt_bbox_labels | wc -l

In [ ]:
import time
import glob
import torch
import os
import PIL

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [ ]:
os.chdir("/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/")

In [ ]:
def are_files_exist(meta_data, files_location):
  not_found_files = []
  file_names = open(meta_data).readlines()
  for file_path in file_names:
    if not os.path.exists(file_path.strip()):
      not_found_files.append(file_path.strip())
  print(f'{len(not_found_files)} number of files not found')
  return not_found_files

In [ ]:
# are_files_exist('/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/data/customdata/train.txt', '/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/inputs/ppe_input_images')

In [ ]:
# are_files_exist('/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/data/customdata/test.txt', '/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/inputs/ppe_input_images')

In [ ]:
def write_image_shapes(image_paths_file, target_filepath):
  file_names = open(image_paths_file).readlines()
  with open(target_filepath, 'a') as file_handler:
    for image_path in file_names:
      im = PIL.Image.open(image_path.strip())
      img_width, img_height = [float(dim) for dim in im.size]
      file_handler.write(f'{img_width} {img_height}\n')
  print("Done")

In [ ]:
# len(open('./data/customdata/custom_train.txt').readlines()), len(open('./data/customdata/custom_train.shapes').readlines())

In [ ]:
# len(open('./data/customdata/custom_test.txt').readlines()), len(open('./data/customdata/custom_test.shapes').readlines())

In [ ]:
# write_image_shapes('./data/customdata/custom_train.txt', './data/customdata/custom_train.shapes')

In [ ]:
# write_image_shapes('./data/customdata/custom_test.txt', './data/customdata/custom_test.shapes')

In [ ]:
def get_device_type():
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  return device

In [ ]:
#weights_yolo = 'weights/yolov3-spp-ultralytics.pt'
#weights_midas = 'weights/model-f46da743.pt'
#device = get_device_type()

In [ ]:
# chkpt_yolo = torch.load(weights_yolo, map_location=device)
# chkpt_midas = torch.load(weights_midas, map_location=device)

In [ ]:
# chkpt_yolo.keys() # it saved as entire model

In [ ]:
#chkpt_midas.keys() # it saved as only weights

In [ ]:
#chkpt_yolo["model"]['module_list.106.BatchNorm2d.weight'].shape

In [ ]:
# for item in chkpt_midas.keys():
#   print(item)
#   print(chkpt_midas[item].size())

In [ ]:
# for item in chkpt_midas.keys():
  # print(item)

# for item in chkpt_yolo["model"].keys():
#   print(item)
#   print(chkpt_yolo["model"][item].size())

## Training yolo branch only by freezing the midas layers on 64x64 resolution

In [ ]:
## Ignore the model summary in the below output - as it is summary on dummy input (3, 416,416) and before freezing the midas layers. But you can observe img_size attribute though as 64.
# Training yolo branch only by freezing the midas layers on 64x64 resolution
!python train.py --data data/customdata/custom.data --batch 32 --cache --cfg cfg/yolov3-custom.cfg --epochs 50 --midas_weights='weights/model-f46da743.pt' --yolo_weights='weights/last_ppe.pt' --init_train='True' --img-size=64 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=32, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=50, evolve=False, img_size=[64], init_train='True', midas_weights='weights/model-f46da743.pt', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='', yolo_weights='weights/last_ppe.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', total_memory=16130MB)

2020-11-13 23:27:02.084096: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 50
batch_size - 32
accumulate - 4
yolo weights - weights/last_ppe.pt
midas weights - weights/model-f46da743.pt
imgsz_min- 64, imgsz_max- 64, imgsz_test- 64
opt.rect - False
train_p

In [ ]:
# !ls /content/drive/My\ Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/ground_truths/depthmap_gt_images/K25.png
!ls /content/drive/My\ Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/ground_truths/planar_surface_gt_images/K25_segmentation_final.png


'/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/ground_truths/planar_surface_gt_images/K25_segmentation_final.png'


In [ ]:
# !python train.py --data data/customdata/custom.data --batch 10 --cache --cfg cfg/yolov3-custom.cfg --epochs 50 --midas_weights='weights/model-f46da743.pt' --yolo_weights='weights/last.pt' --init_train='True' --img-size=64

## Training yolo branch only by freezing the midas layers on 64x64 resolution for few more epochs

In [ ]:
## Ignore the model summary in the below output - as it is summary on dummy input (3, 416,416) and before freezing the midas layers. But you can observe img_size attribute though as 64.
# Training yolo branch only by freezing the midas layers on 64x64 resolution
!python train.py --data data/customdata/custom.data --batch 32  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=64 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=32, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[64], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 05:53:42.864898: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 32
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 64, imgsz_max- 64, imgsz_test- 64
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt
i

## Training yolo branch only by freezing the midas layers on 128x128 resolution

In [ ]:
## Ignore the model summary in the below output - as it is summary on dummy input (3, 416,416) and before freezing the midas layers. But you can observe img_size attribute though as 128.
# Training yolo branch only by freezing the midas layers on 128x128 resolution
!python train.py --data data/customdata/custom.data --batch 16  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=128 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[128], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 10:39:45.785315: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 16
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 128, imgsz_max- 128, imgsz_test- 128
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.t

## Training yolo branch only by freezing the midas layers on 256x256 resolution

In [ ]:
## Correct the model summary in the below output to print on given resolution and after freezing the midas layers. This is perfect model summary
# Training yolo branch only by freezing the midas layers on 256x256 resolution
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=256 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[256], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 11:53:04.210579: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 256, imgsz_max- 256, imgsz_test- 256
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt

## Training yolo branch only by freezing the midas layers on 448x448 resolution

In [ ]:
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=448 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[448], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 14:35:15.043896: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 448, imgsz_max- 448, imgsz_test- 448
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt

## Training yolo branch only by freezing the midas layers on 512x512 resolution

In [ ]:
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=512 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[512], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 15:42:10.266764: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 512, imgsz_max- 512, imgsz_test- 512
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt

In [ ]:
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=256 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[256], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 16:26:57.640865: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 256, imgsz_max- 256, imgsz_test- 256
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt

In [ ]:
!python train.py --data data/customdata/custom.data --batch 16  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=448 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[448], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 16:39:16.272632: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 16
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 448, imgsz_max- 448, imgsz_test- 448
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.t

In [ ]:
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=512 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[512], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 17:25:46.233087: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 512, imgsz_max- 512, imgsz_test- 512
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt

In [ ]:
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=512 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[512], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-14 20:26:13.979827: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 512, imgsz_max- 512, imgsz_test- 512
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt

In [ ]:
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=512 --midasnet_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=300, evolve=False, img_size=[512], init_train='False', midas_weights='', midasnet_freeze='True', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-11-15 08:42:24.436491: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 512, imgsz_max- 512, imgsz_test- 512
opt.rect - False
train_path - data/customdata/train.txt
test_path - data/customdata/test.txt

In [ ]:
!md5sum weights/best.pt
!md5sum weights/last_best_diwali_2am.pt
!md5sum weights/last.pt

4d111871c26592ce86197c2d04fa5087  weights/best.pt
4d111871c26592ce86197c2d04fa5087  weights/last_best_diwali_2am.pt
741f479c7f18431ba15649b7cf6b8ecc  weights/last.pt


In [ ]:
!python train.py --data data/customdata/custom_test.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=448 --midasnet_freeze='False'

Namespace(accumulate=4, adam=False, batch_size=8, bbox_lambda=1, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom_test.data', depth_lambda=1, device='', epochs=300, evolve=False, img_size=[448], init_train='False', midas_weights='', midasnet_freeze='False', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', total_memory=16130MB)

2020-11-18 13:48:54.316535: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom_test.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 448, imgsz_max- 448, imgsz_test- 448
opt.rect - False
train_path - data/customdata/cust

In [ ]:
# !python train.py --data data/customdata/custom_test.data --batch 8 --cache --cfg cfg/yolov3-custom.cfg --epochs 50 --midas_weights='weights/model-f46da743.pt' --yolo_weights='weights/last_ppe.pt' --init_train='True' --img-size=512 --midasnet_freeze='False'

In [ ]:
!md5sum ./weights/last_best_diwali_2am.pt
!md5sum ./weights/best.pt
!md5sum ./weights/last.pt

4d111871c26592ce86197c2d04fa5087  ./weights/last_best_diwali_2am.pt
4d111871c26592ce86197c2d04fa5087  ./weights/best.pt
d0f8da0323a1620f913934168ac0aa79  ./weights/last.pt


In [ ]:
!python train.py --data data/customdata/custom.data --batch 32  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=64 --midasnet_freeze='False'

Namespace(accumulate=4, adam=False, batch_size=32, bbox_lambda=1, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', depth_lambda=1, device='', epochs=300, evolve=False, img_size=[64], init_train='False', midas_weights='', midasnet_freeze='False', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', total_memory=16130MB)

2020-11-18 14:00:56.949556: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 32
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 64, imgsz_max- 64, imgsz_test- 64
opt.rect - False
train_path - data/customdata/train.txt
test_p

In [ ]:
!python train.py --data data/customdata/custom.data --batch 32  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/last.pt'  --img-size=64 --midasnet_freeze='False'

Namespace(accumulate=4, adam=False, batch_size=32, bbox_lambda=1, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', depth_lambda=1, device='', epochs=300, evolve=False, img_size=[64], init_train='False', midas_weights='', midasnet_freeze='False', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/last.pt', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', total_memory=16130MB)

2020-11-18 20:55:59.590448: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 32
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 64, imgsz_max- 64, imgsz_test- 64
opt.rect - False
train_path - data/customdata/train.txt
test_p

## Training MidasNet only with low learning rate 0.0001 on 64x64 resolution input image

In [ ]:
!python train.py --data data/customdata/custom.data --batch 32  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=64 --midasnet_freeze='False' --yolo_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=32, bbox_lambda=1, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', depth_lambda=1, device='', epochs=300, evolve=False, img_size=[64], init_train='False', midas_weights='', midasnet_freeze='False', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_freeze='True', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', total_memory=16130MB)

2020-11-18 21:29:27.030204: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 32
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 64, imgsz_max- 64, imgsz_test- 64
opt.rect - False
train_path - data/customd

## Training MidasNet only on 448x448 resolution input image

In [ ]:
!python train.py --data data/customdata/custom.data --batch 8  --cache --cfg cfg/yolov3-custom.cfg --epochs 300 --weights 'weights/best.pt'  --img-size=448 --midasnet_freeze='False' --yolo_freeze='True'

Namespace(accumulate=4, adam=False, batch_size=8, bbox_lambda=1, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', depth_lambda=1, device='', epochs=300, evolve=False, img_size=[448], init_train='False', midas_weights='', midasnet_freeze='False', multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt', yolo_freeze='True', yolo_weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', total_memory=16130MB)

2020-11-18 22:07:20.385192: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
cfg - cfg/yolov3-custom.cfg
data - data/customdata/custom.data
epochs - 300
batch_size - 8
accumulate - 4
yolo weights - 
midas weights - 
imgsz_min- 448, imgsz_max- 448, imgsz_test- 448
opt.rect - False
train_path - data/custo